In [1]:
import os,pickle
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import torch
from tqdm import tqdm
from transformers import BitsAndBytesConfig,LlamaTokenizer,LlamaForCausalLM 
from Custom_bert_generate2 import CustomDataset, Model,get_generated_sentence_data, get_accuracy_by_bert
from get_model import get_model
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns


model_names = ['LLaMA-2-7B-Chat','LLaMA-3-7B-Instruct','Alpaca-7B',
               'DialoGPT-large', 'GPT-2-large',
               'WizardLM-2-7B','WizardLM-7B','WizardCoder-7B','WizardMath-7B',
               'ChatGLM-6B','Guanaco-2-7B']

model_name = model_names[9] # 7,8,9, 하면 됨 하고 나서 'Guanaco-2-7B'쌓는 부분확인 하기 . 
if model_name in ['DialoGPT-large', 'GPT-2-large']:
    device = 'cpu'
else:
    device = 'cuda'
tokenizer,model = get_model(model_name,device)
model.eval()

ChatGLM-6B


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

The repository for THUDM/chatglm2-6b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/THUDM/chatglm2-6b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in_

- generate 재실험하고 있는것들 (8BIT 양자화 재실험 )
- llama2,4bit랑 성능 차이 거의 안남->4 bit로 바꾸기 (generate 만 바꾸면 됨) 끝
- llama3, : 4bit가 더 성능좋음 ->4 bit로 바꾸기 (generate 만 바꾸면 됨) 끝
- alpaca,  4bit가 더 성능좋음 ->4 bit로 바꾸기 (generate 만 바꾸면 됨) 끝
- Dialo, 대화끼리 띄어놔도 성능 비슷함. 의미 없음 끝
- gpt 끝
- wizard-2- 7B-> 8bit oom뜨니까 4BIT로 그냥놔두기 끝  

- wizardLM-7B 4bit랑 성능 차이 거의 안남->4 bit로 바꾸기 (generate 만 바꾸면 됨) 끝 (server 0)


- 'WizardCoder-7B', 8bit 끝
- 'WizardMath-7B' 8bit로 하는중 (colab1) hidden만들어야됨 (server1)
- 'ChatGLM-6B', 양자화 안하고 (colab2)-hidden만들기 (server3)
- 'Guanaco-2-7B'(server 2 ) -prompt 만들어야됨 hidden만들면됨


In [2]:
def get_data():
    with open('/home/wooseok/peft/FT_input_datasets_llama.pickle', 'rb') as fr:
        data = pickle.load(fr)
    del_index = []
    for i in range(len(data)):
        if len(data['context'][i]) %2 ==0:
            del_index.append(i)
    data.drop(del_index, axis=0, inplace=True)
    data=data.reset_index(drop = True)
    return data
if model_name == 'Guanaco-2-7B':
    def make_prompt(j,attribute,sys_prompt,data):
        default_prompt = "### Instruction:"+"\n"
        sys_prompt = default_prompt+ sys_prompt + attribute+"\n"
        prompt = sys_prompt 
        conversation_history = ''
        for i in range(len(data['context'][j])):
            if i%2 == 0:
                utter2 = 'USER: '+data['context'][j][i]+"\n"
            if i%2 == 1:
                utter2 = 'ASSISTANT: '+data['context'][j][i]
            conversation_history = conversation_history + utter2
        prompt = prompt+"### Input: "+"\n"+conversation_history+"\n### Response:" +"\nASSISTANT: "
        return prompt, conversation_history
if model_name in ['WizardLM-2-7B','WizardLM-7B']:
    def make_prompt(j,attribute,sys_prompt,data):
        default_prompt = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."
        sys_prompt = default_prompt+ sys_prompt + attribute
        prompt = sys_prompt 
        conversation_history = ''
        for i in range(len(data['context'][j])):
            if i%2 == 0:
                utter2 = 'USER: '+data['context'][j][i]
            if i%2 == 1:
                utter2 = 'ASSISTANT: '+data['context'][j][i]+"</s>"
            conversation_history = conversation_history + utter2
        prompt = prompt+conversation_history +"ASSISTANT: "
        return prompt, conversation_history
if model_name in ['DialoGPT-large', 'GPT-2-large']:
    def make_prompt(j,attribute,sys_prompt,data):
        sys_prompt = sys_prompt +"\n" + attribute+"<|endoftext|>"+"\n"
        utter1 = data['context'][j][0] 
        prompt = sys_prompt +'Conversation:'
        conversation_history = ''
        for i in range(len(data['context'][j])):
            utter2 = '\n'+data['context'][j][i] +'<|endoftext|>'
            conversation_history = conversation_history + utter2
        prompt = prompt+conversation_history
        return prompt, conversation_history
if model_name == 'LLaMA-3-7B-Instruct':

    def make_prompt(j,attribute,sys_prompt,data):
        sys_prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"+sys_prompt +attribute
        sys_prompt = sys_prompt +"<|eot_id|><|start_header_id|>user1<|end_header_id|>\n"
        
        conversation_history = ''
        for i in range(len(data['context'][j])):
            if i%2 == 0:
                utter2 = '\n\n'+data['context'][j][i] +'<|eot_id|><|start_header_id|>user2<|end_header_id|>'
            if i%2 == 1:
                utter2 = '\n\n'+data['context'][j][i] +'<|eot_id|><|start_header_id|>user1<|end_header_id|>'
            conversation_history = conversation_history + utter2
        prompt = sys_prompt+conversation_history
        
        return prompt, conversation_history
if model_name == 'LLaMA-2-7B-Chat':
    def make_prompt(j,attribute,sys_prompt,data):    
        sys_prompt = "[INST]<<SYS>>\n"+sys_prompt +"\n<</SYS>>\n" + attribute+"\n"+'Conversation:'
         
        
        conversation_history = ''
        for i in range(len(data['context'][j])):
            utter2 = '\n'+data['context'][j][i] +'</s>'
            conversation_history = conversation_history + utter2
        prompt = sys_prompt+conversation_history +'[/INST]'
        return prompt, conversation_history
if model_name in ['Alpaca-7B','WizardCoder-7B','WizardMath-7B','ChatGLM-6B']:
    def make_prompt(j,attribute,sys_prompt,data):
        default_prompt = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."

        default_prompt = default_prompt +"\n### Instruction:\n"
        sys_prompt = default_prompt + sys_prompt + attribute +"\n"+"\n### Input:"
        
        prompt = sys_prompt
        conversation_history = ''
        for i in range(len(data['context'][j])):
            if i%2 == 0:
                utter2 = '\n'+data['context'][j][i] +'</s>'
            if i%2 == 1:
                utter2 = '\n'+data['context'][j][i] +'</s>'
            conversation_history = conversation_history + utter2
        prompt = prompt+conversation_history +"\n\n### Response:"
        
        return prompt, conversation_history
def prompt_make(attribute,sys_prompt):
    data = get_data()
    
    prompt_gen = []
    history_gen = []
    for j in tqdm(range(len(data))):
        kk = make_prompt(j,attribute,sys_prompt,data)
        prompt_gen.append(kk[0])
        history_gen.append(kk[1])
                
    data['prompt']=prompt_gen
    data['history']=history_gen
    return data

In [3]:

def get_hidden_collections(sys_prompt,attribute,task_name,last):
    data = prompt_make(attribute,sys_prompt)
    
    data = data.sample(frac=1,random_state = 1).reset_index(drop=True)[:data_num]
    hidden_collections = []

    for i in tqdm(range(len(data))):
        encoded_context = torch.tensor([tokenizer.encode(data['prompt'][i])])
        with torch.no_grad():
            outputs = model(encoded_context, output_hidden_states=True,use_cache=True)
        encoded_history=torch.tensor([tokenizer.encode(data['history'][i])])
        if model_name == 'ChatGLM-6B':
            xx  = torch.stack(list(outputs.hidden_states), dim=0).permute(0,2,1, 3)
        else:
            xx  = torch.stack(list(outputs.hidden_states), dim=0)
        if last == True:
            xx = xx[:, 0, -1, :]
        elif last == False:
            if model_name == 'LLaMA-3-7B-Instruct':
                xx = xx[:,:,-len(encoded_history[0]-1):,:] 
            elif model_name == 'LLaMA-2-7B-Chat':
                xx = xx[:,:,-len(encoded_history[0]+2):,:]
            elif model_name in ['Alpaca-7B','WizardCoder-7B','WizardMath-7B']:
                xx = xx[:,:,-len(encoded_history[0]-4):,:]
            elif model_name in ['DialoGPT-large', 'GPT-2-large']:
                xx = xx[:,:,-len(encoded_history[0]):,:]
            elif model_name in ['WizardLM-2-7B','WizardLM-7B']:
                xx = xx[:,:,-len(encoded_history[0])-1:,:]
            elif model_name == 'Guanaco-2-7B':
                xx = xx[:,:,-len(encoded_history[0])-7:,:]
            elif model_name == 'ChatGLM-6B':
                xx = xx[:,:,-len(encoded_history[0])-3:,:]
            
        print(data['prompt'][i])
        print("쌓는 부분 :")
        print(tokenizer.decode(encoded_context[0][-len(encoded_history[0])-4:]))
        print(" ")
        hidden_collections.append(xx)

    return hidden_collections

In [4]:

def make_hidden(sys_prompt,attribute_name_list,task_name,last):
    attribute_hidden_list = []
    for i in attribute_name_list:
        attribute_hidden_list.append(get_hidden_collections(sys_prompt,i,task_name,last))
    if last ==True:
        with open("최종results_{}/hidden_{}.pkl".format(model_name,task_name),"wb") as f:
            pickle.dump(attribute_hidden_list, f)
    elif last == False:
        with open("최종results_{}/평균hidden_{}.pkl".format(model_name,task_name),"wb") as f:
            pickle.dump(attribute_hidden_list, f)
        
    return attribute_hidden_list
def plot_per_layer_cosine(attribute_name_list, attribute_hidden_list,general_hidden_collections,task_name):
    if model_name in ['LLaMA-2-7B-Chat','LLaMA-3-7B-Instruct','Alpaca-7B','WizardLM-2-7B','WizardLM-7B',
                     'WizardCoder-7B','WizardMath-7B','Guanaco-2-7B']:
        num_of_layer = 33
    if model_name in ['DialoGPT-large', 'GPT-2-large']:
        num_of_layer = 37
    if model_name == 'ChatGLM-6B':
        num_of_layer = 29
    results_for_att_avg = torch.zeros_like(torch.empty(num_of_layer))
    for i, attribute_hidden in enumerate(attribute_hidden_list):
        total_cos_per_layers = []
        for k in range(data_num):
            att_hidden = attribute_hidden[k]
            general_hidden = general_hidden_collections[k]
            cos_per_layers = torch.mean(torch.nn.functional.cosine_similarity(att_hidden, general_hidden, dim=-1), axis=-1).squeeze()
            #cos_per_layers = torch.nn.functional.cosine_similarity(att_hidden, general_hidden, dim=-1).squeeze()
            total_cos_per_layers.append(cos_per_layers)
        result = sum(total_cos_per_layers) / data_num
        results_for_att_avg += result
    
    avg_result = results_for_att_avg/len(attribute_hidden_list)
    
    with open("최종results_{}/평균results_plot_{}.pkl".format(model_name,task_name),"wb") as f:
        pickle.dump(avg_result, f)
    return avg_result

In [5]:
data_num = 50
sys_prompt = "In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context."
emotions = ["joy", "surprise", "anger", "fear", "disgust", "sadness"]
attribute_name_list = ["You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is joy.",
                       "You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is surprise.",
                       "You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is anger.",
                       "You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is fear.",
                       "You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is disgust.",
                       "You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is sadness."
                      ]


task_name = 'general'
general_hidden_collections = get_hidden_collections(sys_prompt,"",task_name,last=False)
with open("최종results_{}/평균general.pkl".format(model_name),"wb") as f:
    pickle.dump(general_hidden_collections , f)

general_hidden_collections_last = get_hidden_collections(sys_prompt,"",task_name,last=True)
with open("최종results_{}/general.pkl".format(model_name),"wb") as f:
    pickle.dump(general_hidden_collections_last , f)

  2%|█▍                                                                        | 1/50 [00:01<01:19,  1.62s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.

### Input:
Mom , when you come home , please buy a coat , a skirt , a sweater ... </s>

### Response:
쌓는 부분 :
:
Mom , when you come home , please buy a coat , a skirt , a sweater ... </s>

### Response:
 


  4%|██▉                                                                       | 2/50 [00:02<00:56,  1.17s/it]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.

### Input:
Good afternoon ! Trans Pacific Airline . How can I help you ? </s>
 I ’ m calling to cancel my reservation for Trans Pacific Flight 8 on July 19 from Taipei to Minneapolis . Could you please make it for the same flight on August 2 instead of July 19 ? </s>
 What ’ s your name , please ? </s>

### Response:
쌓는 부분 :
:
Good afternoon ! Trans Pacific Airline . How can I help you ? </s>
 I ’ m calling to cancel my reservation for Trans Pacific Flight 8 on July 19 from Taipei to Minneapolis . Could you please make it for the same flight on August 2 instead of July 19 ? </s>
 What ’ s your name , please ? </s>

### Response:
 


  6%|████▍                                                                     | 3/50 [00:03<00:47,  1.01s/it]

...

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.

### Input:
excuse me , could you tell me which line I ’ m supposed to stand in to buy bubble wrap and to post a package ? </s>
 you can buy the bubble wrap here , but you ’ ll have to stand in line over here to post your passage . </s>
 that ’ s a really long line . How long do you think it ’ ll take to get through all those people ? </s>

### Response:
쌓는 부분 :
:
excuse me , could you tell me which line I ’ m supposed to stand in to buy bubble wrap and to post a package ? </s>
 you can buy the bubble wrap here , but you ’ ll have to stand in line over here to post your passage . </s>
 that ’ s a really long line . How long do you think it ’ ll take to get through all those people ? </s>

 98%|███████████████████████████████████████████████████████████████████████▌ | 49/50 [00:31<00:00,  1.59it/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.

### Input:
I'm really in a flap about the interview . </s>

### Response:
쌓는 부분 :
:
I'm really in a flap about the interview . </s>

### Response:
 


100%|█████████████████████████████████████████████████████████████████████████| 50/50 [00:32<00:00,  1.54it/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.

### Input:
I want to buy a black suit . </s>

### Response:
쌓는 부분 :
:
I want to buy a black suit . </s>

### Response:
 


In [6]:
task_name = 'Emotion_control'
attribute_hidden_list = make_hidden(sys_prompt,attribute_name_list,task_name,last=False)

  2%|█▍                                                                        | 1/50 [00:00<00:36,  1.34it/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is joy.

### Input:
Mom , when you come home , please buy a coat , a skirt , a sweater ... </s>

### Response:
쌓는 부분 :
:
Mom , when you come home , please buy a coat , a skirt , a sweater ... </s>

### Response:
 


  4%|██▉                                                                       | 2/50 [00:01<00:43,  1.11it/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is joy.

### Input:
Good afternoon ! Trans Pacific Airline . How can I help you ? </s>
 I ’ m calling to cancel my reservation for Trans Pacific Flight 8 on July 19 from Taipei to Minneapolis . Could you please make it for the same flight on August 2 instead of July 19 ? </s>
 What ’ s your name , please ? </s>

### Response:
쌓는 부분 :
:
Good afternoon ! Trans Pacific Airline . How can I help you ? </s>
 I ’ m calling to cancel my reservation for Trans Pacific Flight 8 on July 19 from Taipei to Minneapolis . Could you please make it for the same flight on August 2 instead of July 19 ? </s>
 What ’ 

  6%|████▍                                                                     | 3/50 [00:02<00:44,  1.06it/s]

...

 98%|███████████████████████████████████████████████████████████████████████▌ | 49/50 [00:34<00:00,  1.42it/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is sadness.

### Input:
I'm really in a flap about the interview . </s>

### Response:
쌓는 부분 :
:
I'm really in a flap about the interview . </s>

### Response:
 


100%|█████████████████████████████████████████████████████████████████████████| 50/50 [00:35<00:00,  1.41it/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is sadness.

### Input:
I want to buy a black suit . </s>

### Response:
쌓는 부분 :
:
I want to buy a black suit . </s>

### Response:
 


OSError: [Errno 28] No space left on device

In [7]:
attribute_hidden_list_last = make_hidden(sys_prompt,attribute_name_list,task_name,last=True)

  2%|█▍                                                                        | 1/50 [00:00<00:43,  1.14it/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is joy.

### Input:
Mom , when you come home , please buy a coat , a skirt , a sweater ... </s>

### Response:
쌓는 부분 :
:
Mom , when you come home , please buy a coat , a skirt , a sweater ... </s>

### Response:
 


  4%|██▉                                                                       | 2/50 [00:01<00:39,  1.22it/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is joy.

### Input:
Good afternoon ! Trans Pacific Airline . How can I help you ? </s>
 I ’ m calling to cancel my reservation for Trans Pacific Flight 8 on July 19 from Taipei to Minneapolis . Could you please make it for the same flight on August 2 instead of July 19 ? </s>
 What ’ s your name , please ? </s>

### Response:
쌓는 부분 :
:
Good afternoon ! Trans Pacific Airline . How can I help you ? </s>
 I ’ m calling to cancel my reservation for Trans Pacific Flight 8 on July 19 from Taipei to Minneapolis . Could you please make it for the same flight on August 2 instead of July 19 ? </s>
 What ’ 

  6%|████▍                                                                     | 3/50 [00:02<00:35,  1.34it/s]

...

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is sadness.

### Input:
excuse me , could you tell me which line I ’ m supposed to stand in to buy bubble wrap and to post a package ? </s>
 you can buy the bubble wrap here , but you ’ ll have to stand in line over here to post your passage . </s>
 that ’ s a really long line . How long do you think it ’ ll take to get through all those people ? </s>

### Response:
쌓는 부분 :
:
excuse me , could you tell me which line I ’ m supposed to stand in to buy bubble wrap and to post a package ? </s>
 you can buy the bubble wrap here , but you ’ ll have to stand in line over here to post your passage . </s>

 98%|███████████████████████████████████████████████████████████████████████▌ | 49/50 [00:36<00:00,  1.34it/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is sadness.

### Input:
I'm really in a flap about the interview . </s>

### Response:
쌓는 부분 :
:
I'm really in a flap about the interview . </s>

### Response:
 


100%|█████████████████████████████████████████████████████████████████████████| 50/50 [00:37<00:00,  1.34it/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
In this task you will be shown a conversation context. You need to generate a response to the conversation based on the context.You should generate an utterance that reflects the speaker's emotion given. Speaker's emotion is sadness.

### Input:
I want to buy a black suit . </s>

### Response:
쌓는 부분 :
:
I want to buy a black suit . </s>

### Response:
 


In [8]:
plot_per_layer_cosine(attribute_name_list, attribute_hidden_list,general_hidden_collections,task_name)

tensor([1.0000, 0.9995, 0.9995, 0.9995, 0.9995, 0.9995, 0.9990, 0.9990, 0.9982,
        0.9979, 0.9975, 0.9910, 0.9887, 0.9822, 0.9802, 0.9762, 0.9746, 0.9692,
        0.9676, 0.9657, 0.9647, 0.9610, 0.9583, 0.9569, 0.9530, 0.9502, 0.9473,
        0.9469, 0.9266])